In [1]:
import pandas as pd
from pathlib import Path
import os
import sys
import seaborn as sns

In [2]:
PROJECT_ROOT = Path(os.getcwd()).resolve().parents[0]

In [3]:
def combine_excel_sheets(path_in: str) -> pd.DataFrame:
    """
    Read all sheets from an Excel file, using the first row as headers,
    add a 'City' column with the sheet name, and vertically concatenate.
    """
    try:
        xls = pd.ExcelFile(path_in)
    except Exception as e:
        sys.exit(f"Failed to open Excel file: {e}")

    frames = []
    for sheet in xls.sheet_names:
        # Read sheet with first row as header
        df = pd.read_excel(xls, sheet_name=sheet, header=0)

        # Drop rows that are completely empty (optional but useful)
        df = df.dropna(how="all")

        # Normalize column names a bit (optional)
        df.columns = [str(c).strip() for c in df.columns]

        # Add the City column with sheet name
        df.insert(0, "City", sheet)

        # Skip if truly empty after cleanup
        if not df.empty:
            frames.append(df)

    if not frames:
        sys.exit("No usable data found in any sheet.")

    combined = pd.concat(frames, ignore_index=True)
    return combined

In [4]:
df = combine_excel_sheets(PROJECT_ROOT / "data/raw/daily_weather.xlsx")

In [5]:
df["City"].unique()

array(['OUJDA', 'FES', 'MEKNES', 'MIDELT', 'ERRACHIDIA', 'NADOR',
       'TETOUAN', 'TANGER', 'LARACHE', 'KENITRA', 'SALE', 'LAAYOUNE',
       'DAKHLA', 'AGADIR', 'OUARZAZATE', 'GUELMIM', 'SMARA', 'MARRAKECH',
       'ESSAOUIRA', 'BENI MELLAL', 'SAFI', 'KHOURIBGA', 'SETTAT',
       'EL JADIDA', 'CASABLANCA'], dtype=object)

In [6]:
city_to_region = {
    'OUJDA': 'Oriental',
    'NADOR': 'Oriental',
    'FES': 'Fès-Meknès',
    'MEKNES': 'Fès-Meknès',
    'MIDELT': 'Drâa-Tafilalet',
    'ERRACHIDIA': 'Drâa-Tafilalet',
    'TETOUAN': 'Tanger-Tétouan-Al Hoceïma',
    'TANGER': 'Tanger-Tétouan-Al Hoceïma',
    'LARACHE': 'Tanger-Tétouan-Al Hoceïma',
    'KENITRA': 'Rabat-Salé-Kénitra',
    'SALE': 'Rabat-Salé-Kénitra',
    'LAAYOUNE': 'Laâyoune-Sakia El Hamra',
    'DAKHLA': 'Laâyoune-Sakia El Hamra',
    'AGADIR': 'Souss-Massa',
    'OUARZAZATE': 'Drâa-Tafilalet',
    'GUELMIM': 'Laâyoune-Sakia El Hamra',
    'SMARA': 'Laâyoune-Sakia El Hamra',
    'MARRAKECH': 'Marrakech-Safi',
    'ESSAOUIRA': 'Marrakech-Safi',
    'BENI MELLAL': 'Béni Mellal-Khénifra',
    'KHOURIBGA': 'Béni Mellal-Khénifra',
    'SAFI': 'Marrakech-Safi',
    'SETTAT': 'Casablanca-Settat',
    'EL JADIDA': 'Casablanca-Settat',
    'CASABLANCA': 'Casablanca-Settat'
}


In [7]:
df["Region"] = df["City"].apply(lambda x: city_to_region[x])

In [8]:
df

,City,date,temperature_2m_mean,temperature_2m_max,temperature_2m_min,apparent_temperature_mean,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,...,wind_speed_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,et0_fao_evapotranspiration_sum,wet_bulb_temperature_2m_mean,vapour_pressure_deficit_max,soil_moisture_0_to_100cm_mean,soil_temperature_0_to_100cm_mean,Region
0,OUJDA,2013-01-01,7.943916,10.458500,5.058500,5.543375,7.919854,2.501511,7992.830566,35549.746094,...,11.808878,209.957932,3.550000,0.678337,0.678337,6.363583,0.406565,0.296602,10.800855,Oriental
1,OUJDA,2013-01-02,8.402249,11.408501,5.258500,6.828894,9.582191,3.692343,26197.000000,35586.101562,...,10.464797,277.509216,6.840000,0.875720,0.875720,7.591252,0.223625,0.299802,10.883709,Oriental
2,OUJDA,2013-01-03,6.939749,10.558500,4.008500,4.610157,6.842862,2.134888,30648.066406,35625.261719,...,16.039202,48.863884,8.780000,1.168056,1.168056,5.545732,0.508350,0.300722,10.632083,Oriental
3,OUJDA,2013-01-04,6.979333,12.908501,1.158500,4.983939,11.160752,-1.470960,31857.753906,35667.183594,...,7.208994,152.622421,12.150000,1.577554,1.577554,5.183821,0.643040,0.300097,10.334105,Oriental
4,OUJDA,2013-01-05,7.456417,14.808500,1.908500,4.734254,12.155128,-1.539309,32161.345703,35711.804688,...,10.105681,192.317474,12.340000,1.718802,1.718802,4.733588,0.960209,0.298893,10.150168,Oriental
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116870,CASABLANCA,2025-10-15,19.733665,22.954500,17.604500,21.244984,24.719969,18.744045,30763.460938,40888.453125,...,15.948040,254.814957,16.209999,2.736556,2.736556,18.062006,0.981068,0.138367,24.134834,Casablanca-Settat
116871,CASABLANCA,2025-10-16,20.352415,25.254499,17.504499,21.831911,25.854872,18.657412,37339.242188,40769.511719,...,14.028457,283.828644,17.520000,3.023045,3.023045,18.015036,1.663147,0.138457,24.101355,Casablanca-Settat
116872,CASABLANCA,2025-10-17,20.429499,23.404501,18.354500,22.870359,25.355989,20.544962,34610.531250,40650.476562,...,12.245293,336.587830,16.040001,2.717700,2.717700,19.058550,0.768465,0.138335,24.051147,Casablanca-Settat
116873,CASABLANCA,2025-10-18,20.021166,23.904501,17.304501,21.913538,26.597141,18.689278,30324.240234,40531.457031,...,9.546790,279.462250,14.320000,2.492933,2.492933,18.510748,0.816383,0.139175,23.946375,Casablanca-Settat


In [10]:
df["date"] = pd.to_datetime(df["date"])

In [13]:
df["Year"] = df["date"].dt.year
df["Month"] = df["date"].dt.month

In [18]:
df.columns

Index(['City', 'date', 'temperature_2m_mean', 'temperature_2m_max',
       'temperature_2m_min', 'apparent_temperature_mean',
       'apparent_temperature_max', 'apparent_temperature_min',
       'sunshine_duration', 'daylight_duration', 'precipitation_sum',
       'precipitation_hours', 'wind_speed_10m_max',
       'wind_direction_10m_dominant', 'shortwave_radiation_sum',
       'et0_fao_evapotranspiration', 'et0_fao_evapotranspiration_sum',
       'wet_bulb_temperature_2m_mean', 'vapour_pressure_deficit_max',
       'soil_moisture_0_to_100cm_mean', 'soil_temperature_0_to_100cm_mean',
       'Region', 'Year', 'Month'],
      dtype='object')

In [31]:
df.to_csv(PROJECT_ROOT / "data/daily_meteo_2013_2025.csv")

In [19]:
df_bt = pd.read_csv(PROJECT_ROOT / "data/srm_data_2007_2023_bt.csv")

In [ ]:
df_bt.columns # 'tempmax', 'tempmin', 'temp', 'precip'

Index(['Classe', 'Activity', 'City', 'Year', 'Month', 'Nbr Clients', 'MWh',
       'kDH', 'tempmax', 'tempmin', 'temp', 'precip', 'Region'],
      dtype='object')

In [36]:
df.drop(columns = ["City"], inplace = True)

In [37]:
df.columns

Index(['date', 'temperature_2m_mean', 'temperature_2m_max',
       'temperature_2m_min', 'apparent_temperature_mean',
       'apparent_temperature_max', 'apparent_temperature_min',
       'sunshine_duration', 'daylight_duration', 'precipitation_sum',
       'precipitation_hours', 'wind_speed_10m_max',
       'wind_direction_10m_dominant', 'shortwave_radiation_sum',
       'et0_fao_evapotranspiration', 'et0_fao_evapotranspiration_sum',
       'wet_bulb_temperature_2m_mean', 'vapour_pressure_deficit_max',
       'soil_moisture_0_to_100cm_mean', 'soil_temperature_0_to_100cm_mean',
       'Region', 'Year', 'Month'],
      dtype='object')

In [39]:
len(df)

116875

In [40]:
cols_max = ['precipitation_hours', 'Year', 'Month']  # columns for which you want max
agg_dict = {col: 'max' for col in cols_max}

# all other numeric columns get 'mean'
for col in df.columns:
    if col not in ['date', 'Region'] + cols_max:
        agg_dict[col] = 'mean'

result = df.groupby(['date', 'Region'], as_index=False).agg(agg_dict)

In [44]:
result.to_csv(PROJECT_ROOT / "data/daily_meteo_by_region_2013_2025.csv", index = False)

In [3]:
df = pd.read_csv(PROJECT_ROOT / "data/daily_meteo_by_region_2013_2025.csv")

In [5]:
import sqlite3
db_orig = sqlite3.connect(PROJECT_ROOT / "data/ONEE_Regional_COMPLETE_2007_2023.db")

In [6]:
df.to_sql("weather_data", db_orig, if_exists="replace", index=False)

46750